# baseline result fusion

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import hiddenlayer as hl
import torch
import torch.nn as nn
from torch.optim import SGD,Adam
import torch.utils.data as Data
from torchvision import models
from  torchvision import transforms
from  torchvision.datasets import ImageFolder
import pickle as pkl
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
vgg16 = models.vgg16(pretrained=True)
#获取VGG16的特征提取层
vgg = vgg16.features
feature = list(vgg)[:30]
#获取VGG16的分类层
classify = vgg16.classifier
classifier = list(classify)

In [3]:
for layer in feature[:24]:
    for p in layer.parameters():
        p.requires_grad = False

In [4]:
vgg16_1 = nn.Sequential(*feature)

In [5]:
class MyVggModel(nn.Module):
    def __init__(self):
        super(MyVggModel,self).__init__()
        #预训练的Vgg16的特征提取层
        self.vgg_MRI = vgg16_1
        self.vgg_PET = vgg16_1
        #添加新的全连接层
        self.classifier_MRI = nn.Sequential(
            nn.Linear(304128,256),
            nn.ReLU(),
            nn.Dropout(p = 0.5),
            nn.Linear(256,128),
            nn.ReLU(),
            nn.Dropout(p = 0.5),
            #nn.Linear(128,3)

        )
        self.classifier_PET = nn.Sequential(
            nn.Linear(304128,512),
            nn.ReLU(),
            nn.Dropout(p = 0.5),
            nn.Linear(512,256),
            nn.ReLU(),
            nn.Dropout(p = 0.5),
            nn.Linear(256,128),
            nn.ReLU(),
            nn.Dropout(p = 0.5),
            #nn.Linear(128,3)

        )
        self.fc_comb = nn.Sequential(
            nn.Linear(256,3)
        )
        
    def MA(self, x, label):
        # x (k, v), label (q)
        B, C_kv = x.shape
        B, C_q = label.shape
        self.kv = nn.Linear(C_kv, C_kv * 3 * 2).cuda()
        self.q = nn.Linear(C_q, C_kv * 3).cuda()
        self.at_fx = nn.Linear(C_kv * 3, C_kv).cuda()
        #self.ffn = nn.Linear(C_kv, C_kv).cuda()
        kv = self.kv(x).reshape(2, B, 3, C_kv)
        k, v = kv[0], kv[1]
        q = self.q(label).reshape(B, 3, C_kv)
        attn = torch.einsum("bhq,bhk->bhqk", [q, k])
        attn = attn.softmax(dim=-1)
        x_ = torch.einsum("bhqk,bhk->bhq", [attn, v])
        x_ = x_.reshape(B, C_kv * 3)
        x = self.at_fx(x_) + x
        #x = self.ffn(x) + x
        return x    
        
        
    #定义网络的前向传播
    def forward(self,MRI,PET):
        MRI = self.vgg_MRI(MRI)
        PET = self.vgg_PET(PET)
        MRI = MRI.view(MRI.size(0),-1)
        PET = PET.view(PET.size(0),-1)# 将第二次卷积的输出拉伸为一行
        MRI = self.classifier_MRI(MRI)
        PET = self.classifier_PET(PET)
        
        MRI_ma = self.MA(MRI,PET)
        PET_ma = self.MA(PET,MRI)
        concat = torch.cat((MRI_ma, PET_ma), 1)
        output = self.fc_comb(concat)
        return output

In [6]:
Myvggc = MyVggModel()

In [7]:
Myvggc

MyVggModel(
  (vgg_MRI): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dila

In [8]:
#定义优化器
optimizer = torch.optim.Adam(Myvggc.parameters(),lr=0.00001,weight_decay=0.01)
loss_func = nn.CrossEntropyLoss()#损失函数

In [9]:
#记录训练过程指标
historyl = hl.History()
#使用Canves进行可视化

canvasl = hl.Canvas()

In [10]:
from torch.utils.data import DataLoader
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, root):
        super(MyDataset, self).__init__()
        MRI_PET_match_all = pkl.load(open(root,"rb"),encoding='iso-8859-1')
        MRI = []
        PET = []
        group = []
        for index,row in MRI_PET_match_all.iterrows():
            MRI.append(row['MRI_img_array'])
            PET.append(row['PET_img_array'])
            group.append(row['Group'])
        self.MRI = MRI
        self.PET = PET
        self.group = group  

    def __getitem__(self, index):
        mri =torch.from_numpy(self.MRI[index].transpose([2,0,1])).float().to(DEVICE)
        pet = torch.from_numpy(self.PET[index].transpose([2,0,1])).float().to(DEVICE)
        group = self.group[index]
        
        return mri,pet,group

    def __len__(self):
        return len(self.MRI)

train_data = MyDataset("/home/gc/gechang/gec_multi_fusion/end_to_end/train.pkl")
test_data = MyDataset("/home/gc/gechang/gec_multi_fusion/end_to_end/test.pkl")

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32)


In [11]:
#对模型进行迭代训练，对所有的数据训练epoch轮
for epoch in range(100):
    train_loss_epoch = 0
    val_loss_epoch = 0
    train_corrects = 0
    val_corrects = 0
    #对训练数据的加载器进行迭代计算
    Myvggc.train().cuda()
    for step,(mri,pet,group) in enumerate(train_loader):
        ##计算每个batch的损失
        output = Myvggc(mri,pet)
        loss = loss_func(output,group.to(DEVICE))#交叉熵损失函数
        pre_lab = torch.argmax(output,1).to(DEVICE)
        optimizer.zero_grad()#每个迭代步的梯度初始化为0
        loss.backward()#损失的后向传播，计算梯度
        optimizer.step()#使用梯度进行优化
        train_loss_epoch += loss.item()*group.size(0)
        train_corrects += torch.sum(pre_lab == group.to(DEVICE).data)
    #计算一个epoch的损失和精度
    train_loss = train_loss_epoch/len(train_data.group)
    train_acc = train_corrects.double()/len(train_data.group)
    #print("epoch:",epoch,"train_loss:",train_loss,"train_acc:",train_acc)
     #计算在验证集上的表现
    Myvggc.eval()
    for step,(mri,pet,group) in enumerate(test_loader):
        output = Myvggc(mri,pet)
        loss = loss_func(output,group.to(DEVICE))
        pre_lab = torch.argmax(output,1).to(DEVICE) #返回指定维度最大值的序号下标
        val_loss_epoch += loss.item()*group.size(0)
        val_corrects += torch.sum(pre_lab == group.to(DEVICE).data)

    #计算一个epoch上的输出loss和acc
    val_loss = val_loss_epoch/len(test_data.group)
    val_acc = val_corrects.double()/len(test_data.group)
    print("epoch:",epoch,"val_loss:",val_loss,"val_acc:",val_acc)
    #保存每个epoch上的输出loss和acc
    historyl.log(epoch,train_loss=train_loss,val_loss = val_loss,train_acc = train_acc.item(),val_acc = val_acc.item())
    #可视化网络训练的过程
    # with canvasl:
    #     canvasl.draw_plot([historyl["train_loss"],historyl["val_loss"]])
    #     canvasl.draw_plot([historyl["train_acc"],historyl["val_acc"]])


epoch: 0 val_loss: 2.4408586120605467 val_acc: tensor(0.4100, device='cuda:0', dtype=torch.float64)
epoch: 1 val_loss: 1.2053029108047486 val_acc: tensor(0.5450, device='cuda:0', dtype=torch.float64)
epoch: 2 val_loss: 0.9812585115432739 val_acc: tensor(0.5500, device='cuda:0', dtype=torch.float64)
epoch: 3 val_loss: 0.9726630401611328 val_acc: tensor(0.5400, device='cuda:0', dtype=torch.float64)
epoch: 4 val_loss: 0.9679617261886597 val_acc: tensor(0.5450, device='cuda:0', dtype=torch.float64)
epoch: 5 val_loss: 0.953515830039978 val_acc: tensor(0.5550, device='cuda:0', dtype=torch.float64)
epoch: 6 val_loss: 0.9393130493164062 val_acc: tensor(0.5450, device='cuda:0', dtype=torch.float64)
epoch: 7 val_loss: 0.9179563617706299 val_acc: tensor(0.5800, device='cuda:0', dtype=torch.float64)
epoch: 8 val_loss: 0.9025744390487671 val_acc: tensor(0.5750, device='cuda:0', dtype=torch.float64)
epoch: 9 val_loss: 0.8734713053703308 val_acc: tensor(0.5950, device='cuda:0', dtype=torch.float64)
e

: 

In [2]:
x = torch.zeros(2, 1, 2, 1, 2)

In [5]:
x.size()

torch.Size([2, 1, 2, 1, 2])

In [23]:
y = torch.squeeze(x,1) #只有维数为1的能删，别的不能删


In [24]:
y.size()

torch.Size([2, 2, 1, 2])